# Введение в анализ текстов: классификация

-------------------------------

На сегодняшней паре мы будем с вами обучать на текстах свой собственный классификатор. 


##  Анализ тональности общественного мнения

Давайте попробуем сделать что-нибудь прикольное! Например, проанализировать общественное мнение по поводу чего-нибудь. Одним из способов анализа общественного мнения является анализ тональности Twitter по нескольким релевантным хэштегам. Например, вот в [этой статье на Хабре](https://habr.com/company/dca/blog/274027/) пацаны пытались проанализировать динамику общественного мнения о новом эпизоде звёздных войн. 

Попробуем сделать что-то похожее. Для этого возьмём из интернета [готовую разметку твиттера](http://study.mokoron.com) на положительный и отрицательный сентимент-окрас твиттов. На основе этой разметки мы обучим свою собственную модель для классификации твиттов, а после будем применять её. 

In [1]:
import pandas as pd  # для таблиц
import numpy as np   # для матриц

# визуализация 
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

##########################################################
# Любые ваши библиотеки, которые могли бы нам помочь! ####
##########################################################



In [4]:
from visual import ppilif_visual as my_vis

## 1. Данные

Подгрузим данные и полюбуемся на них. 

In [5]:
df_neg = pd.read_csv('twitter_datasets/negative.csv', sep=';',header=-1)
df_pos = pd.read_csv('twitter_datasets/positive.csv', sep=';',header=-1)

df = df_pos[[3,4]].append(df_neg[[3,4]])
df.columns = ['text', 'target']
df.reset_index(drop=True, inplace=True)
df.target = df.target.replace({-1:0})

print(df.shape)
df.head()

(226834, 2)


text  target
0  @first_timee хоть я и школота, но поверь, у на...       1
1  Да, все-таки он немного похож на него. Но мой ...       1
2  RT @KatiaCheh: Ну ты идиотка) я испугалась за ...       1
3  RT @digger2912: "Кто то в углу сидит и погибае...       1
4  @irina_dyshkant Вот что значит страшилка :D\nН...       1

В первом столбце тексты твиттов. Во втором столбце две метки: либо $1$, если твит позитивный, либо $0$, если твит негативный.

In [6]:
df.text.iloc[10]  # пример позитива

'Люблю маму и папу!!!!а в остальное я так...-влюбляюсь, привязываюсь)))\xa0'

In [7]:
df.text.iloc[-10] # пример негатива

'@ivanenko14 и у меня также, только будильник еще и не выключался.. папу разбудила ('

## 2. Предобработка данных 

Начнём с предобработки данных. Напишем для этого классную функцию. 

In [ ]:
def prepare_text(text, stop = stopwords, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    
    # Ваш код, не забудьте привести все тексты к нижнему регистру
    # Перед функцией подгрузите все необходимые библиотеки
    
    return ' '.join(words)


In [ ]:
# Убедитесь на нескольких примерах, что ваша функция работает

Отлично! Если ты это читаешь, у тебя всё работает. Если не работает, прекрати читать! 

Теперь давайте запустим предобработчик на всём нашем корпусе из текстов. Лемматизатор обычно работает довольно долго. Если корпус из текстов на вход идёт довольно большой, приходится паралелить вычисления. Тут мы именно это и сделаем.

In [ ]:
# Библиотека для распараллеливания кода
from joblib import Parallel, delayed
from tqdm import tqdm_notebook

texts = df.text.get_values() 

n_jobs = -1 # параллелим на все ядра 
texts_lemm = Parallel(n_jobs=n_jobs)(delayed(prepare_text)(
    text) for text in tqdm_notebook(texts))

Посмотрим на пример предобработанного текста.

In [ ]:
texts_lemm[10]

In [ ]:
texts_lemm[-10]

## 3. Первые модели

Разбьём выборку на тренировочную и тестовую.

In [ ]:
from sklearn.cross_validation import train_test_split

texts_train, texts_test, X_train, X_test, \
y_train, y_test = train_test_split(texts, texts_lemm, df.target.get_values(), test_size=0.2)

Достаём из зашагников `CountVectorizer` и обучаем первый вариант модели. Будем выстраивать обучение в виде пайалайна. 

In [8]:
from sklearn.feature_extraction.text import  CountVectorizer
count_vectorizer = CountVectorizer()

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Собираем модель из двух шагов (cv - count_vectorizer)
model_cv = Pipeline([
            ('vectorizer', CountVectorizer()),
            ('classifier', LogisticRegression(C=1))
            ])

model_cv.fit(X_train, y_train)

Поизучаем разные куски нашей получившейся модели.

In [ ]:
count_vectorizer = 

# имена фичей
feature_names = count_vectorizer.get_feature_names()

# смотрим на размерность словаря и ужасаемся 
print(len(count_vectorizer.vocabulary_), "words")

Посмотрим на качество получившегося прогноза и итоговые коэффициенты. 

In [ ]:
# Тут подгрузить мои пиздатые функции из резервного файлика 

Теперь давайте попробуем обучить такую же модель но на текстах без предобработки.

In [ ]:
# Собираем модель из двух шагов (cv - count_vectorizer)
model_noprepe = Pipeline([
            ('vectorizer', CountVectorizer()),
            ('classifier', LogisticRegression(C=1))
            ])

model_noprepare.fit(text_train, y_train)

In [ ]:
# Cравниваем модели на качество между собой моей пиздатой картинкой

## 4. Пытаемся улучшить модель

**Задание 1:** Модель вышла не самой удачной. Давайте попробуем обучить точно такую же модель, но с tf-idf векторизацией. Отфильтруем из векторайзера все очень частые и редкие слова так, чтобы фичей осталось не очень много. На самом деле параметры для фильтрации это гипер-параметры и их тоже можно подбирать в ходе поиска по решётке. Также как и силу регуляризации. 

In [ ]:
# Ваша tf-idf модель, которая побила предыдущую модель

Теперь попробуйте добавить в модель в рамках tf-idf биграммы. Приводит ли это к улучшению модели? 

In [ ]:
# Ваша tf-idf модель c биграммами

Визуализируйте топ-10 положительных коэффициентов и топ-10 отрицательных. Логичные ли получились коэффициенты? 

In [ ]:
# ваш код

## 5. Новый пайплайн

На лекции мы с вами выяснили, что главная беда анализа текстов - высокая размерность матрицы термы-на-документы. Давайте попробуем эту беду побороть. Для этого вспомним метод главных компонент, который позволяет сжать пространство высокой размерность во что-то более маленькое и приятное. Используйте для сжатия `TruncatedSVD`. Это реализация PCA, которая работает для разряженных матриц. Нужно добавить SVD в наш вычислительный пайплайн в качестве отдельного шага. 

In [ ]:
# Ваш код 

## 6. Применяем модель.

Выбирете лучшую из своих моделей. Сейчас мы попытаемся применить её на практике. Делай раз. Подбираем порог для позитива и негатива. 

In [ ]:
# Ваш код 

Делай два. Подгружаем новую табличку с твиттами и прогоняем по ней нашу модель. 

In [ ]:
def table_prepare(path, model=model):

    df = pd.read_csv(path, sep='\t')
    df = df[['Tweets', 'Date']]

    # предобработали дату
    df.Date = df.Date.apply(lambda x: x.split(' ')[2] + ' ' 
                        + x.split(' ')[1] + ' ' + x.split(' ')[-1])

    # по очереди применяем все предобрабатывающие функции
    df['prepareTweets'] = df.Tweets.apply(prepare_text)

    print("Размер таблицы: ", df.shape)
    # финальная предобработка (добавление нулей)
    X = df.prepareTweets.values

    # предсказываем вероятность негатива
    prob = model.predict_proba(X)[:,-1]
    df['prob'] = prob
    return df[['Date', 'Tweets', 'prob']]

Делай три. Рисуем весёлую картинку. 

In [ ]:
def negative_plot(df, cutoff_neg=0.65, cutoff_pos=0.45):
    df['Negative'] = df.prob < cutoff_pos
    df['Positive'] = df.prob > cutoff_neg
    df['Neutral'] = (df.prob <= cutoff_neg) | (df.prob >= cutoff_pos)

    df_abs = df[['Date', 'Positive', 'Neutral','Negative']].groupby('Date').sum()
    df_perc = df_abs.divide(df_abs.sum(axis=1), axis=0)
    
    # строим красивую картинку 
    plt.figure(figsize=(14,6))

    # colors: https://www.color-hex.com/color/2ecc71
    pal = ["#e74c3c", "#f1c40f", "#2ecc71"]

    plt.stackplot(df_perc.index, df_perc['Negative'],  df_perc['Neutral'], df_perc['Positive'], 
                        labels=['Negative','Neutral','Positive'],  colors=pal)

    plt.legend(loc='lower right')
    plt.margins(0,0)
    plt.title('Доли твитов определённой тональности',size=18)
    plt.show()
    pass 

Поихали. 

### Что люди пишут в твиттере о кино?

In [ ]:
df_rapsodia = table_prepare('twitter_datasets/df_film_rapsodia.csv')
df_rapsodia.head()

In [ ]:
negative_plot(df_rapsodia)

In [ ]:
df_fantastic = table_prepare('twitter_datasets/df_film_fantastic.csv')
df_fantastic.head()

In [ ]:
negative_plot(df_fantastic)

In [ ]:
# пример позитива
df_fantastic[df_fantastic.prob > 0.9].Tweets.iloc[0]

In [ ]:
# пример негатива
df_fantastic[df_fantastic.prob < 0.1].Tweets.iloc[0]

### Что люди пишут в твиттере о банках?

In [ ]:
df_sber = table_prepare('twitter_datasets/df_sber.csv')
df_sber.head()

In [ ]:
negative_plot(df_sber)

In [ ]:
df_tinkoff = table_prepare('twitter_datasets/df_tinkoff.csv')
df_tinkoff.head()

In [ ]:
negative_plot(df_tinkoff)

### Что люди пишут в твиттере о рэпе?

In [ ]:
df_basta = table_prepare('df_basta.csv')
df_basta.head()

In [ ]:
negative_plot(df_basta)

In [ ]:
df_noize = table_prepare('df_noize.csv')
df_noize.head()

In [ ]:
negative_plot(df_noize)

In [ ]:
df_oxxy = table_prepare('df_oxxy.csv')
df_oxxy.head()

In [ ]:
negative_plot(df_oxxy)

### Ваши идеи

In [ ]:
# ?????

## 7. Эксперименты с деревьями 

На лекции мы обсудили, что деревья, будстинг и тп не очень заходят для решения задач, связаннх с текстами. Давайте убедимся, что они и правда работают хуже. Для экспериментов возьмём небольшой кусочек от базовой выборки. 

Пришло време деревьев. Подгружаем классифайер для случайного леса и бустинг. 

In [98]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

Соберите два пайплайна: `CountVectorizer() -> TruncatedSVD(100) -> Randomforest(100)` и такой же с бустингом. Насколько высоким оказывается качество? 

In [ ]:
# Ваш код

Увеличьте количество компонент и деревьев до $1000$. Стало ли лучше? 

In [ ]:
# Ваш код

Попробуйте взять в качестве векторайзера tf-idf. Удалось ли улучшить результат? 

In [ ]:
# Ваш код

Выберите лучшую модель с деревьями. Бьёт ли она результаты вашей лучшей регрессии? Почему при сопостовимом качестве моделей имеет смысл остановиться на регрессии и не трогать деревья? 

In [ ]:
# Ваш код

## 8.  Наивный байес 

Иногда одним из самых первых алгоритмов, среди натравливаемых на тексты на равне с логистической регрессией, оказывается наивный байесовский классификатор. 

Требуется оценить вероятность принадлежности документа $d \in D$ классу $c \in C$: $p(c|d)$. Каждый документ –  мешок слов, всего слов $|V|$.
	
* $p(c)$ – априорная вероятность класса $c$
* $p(c|d)$ – апостериорная вероятность класса $c$
* $ p(c|d) = \frac{p(d|c)p(c)}{p(d)} $


В мультиномиальной байесовской модели документ – это последовательность событий. Каждое событие – этослучайный выбор одного слова из мешка слов. Когда мы подсчитываем правдоподобие документа, мы перемножаем вероятности того, что мы достали из мешка те самые слова, которые встретились в документе. 

Наивное предположение в том, что мы достаём из мешка разные слова независимо друг от друга, т.е. вероятности признаков внутри класса независимы.

Получается мультиномиальная генеративная модель, которая учитывает количество повторений каждого слова, но не учитывает порядок этих слов, а также каких слов нет в документе.

* [Подробнее о различных видах байесовских классификаторов](https://logic.pdmi.ras.ru/~sergey/teaching/mlaptu11/03-classifiers.pdf).
* [Слайды Дмитрия Игнатова про классификацию](https://cs.hse.ru/data/2016/04/13/1129765566/Classification.pdf)

-------------------------

Давайте попробуем обучить наивную Байесовскую модель. Сделаем это точно также, как и с предыдущими моделями, в виде пайплайна. В качестве первого шага пайплайна попробуйте оба векторайзера по очереди. 

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Ваш код

## 9. Сравниваем всё, что намоделировали!

Постройте все получившиеся precision-recall кривые на одной картинке. Какая самая классная f-мера получилась? Имейте в виду, что разметка твиттера, которой мы пользуемся, была сделана в полуавтоматическом режиме. Сама автор разметки оценивает её качество на уровне 80%. Выборка сбалансированная. Обычно получается, что accuracy на ней в районе $0.75$. Если вы добрались до такой отметины, ваша модель хороша. Если вы пробили 0.8, то вы умудрились оверфитнутся. Поздравляю. 

In [ ]:
# Ваш код

Осталось время? (маловероятно). Тогда попробуем трюк с хэшированием м обсудим, почему, на мой взгляд, его никогда не надо пробовать.

![](https://hsto.org/webt/gw/-l/bs/gw-lbsso67kzbcmb8oibvw-pn1u.png)